In [1]:
import torch
from torch.utils.data import Dataset, DataLoader 
import torch.nn as nn
from torchinfo import summary 
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
import os

In [2]:
# Utilisation du GPU si disponible
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Dataset personnalisé avec les outputs de l'avant dernière couche du RESNET50 pour les images du dataset de train ImageNet
class PenultimateOutputsDataset(Dataset):
    def __init__(self, folder_path, device='cpu'):
        with open(f'{folder_path}/penultimate_layer_outputs/data.json') as data:
            self.data_tensors = json.load(data)
        self.file_list = [f'{folder_path}/penultimate_layer_outputs/penultimate_layer_outputs_{index}.pt' for index in range(self.data_tensors['last_file_indice']+1)]
        self.labels = torch.load(f'{folder_path}/penultimate_layer_labels.pt', map_location=device)
        self.length = self.data_tensors['min_size_segmentation']*self.data_tensors['last_file_indice']+torch.load(f'{folder_path}/penultimate_layer_outputs/penultimate_layer_outputs_{self.data_tensors["last_file_indice"]}.pt', map_location=device).shape[0]
        self.device = device
        self.data_cache = None
        
    def __len__(self):
        return self.length

    def __getitem__(self, index):
        if index < 0 or index >= self.length :
           raise ValueError(f'Index need to be between 0 and {self.length-1}')
        index_file = index//self.data_tensors['min_size_segmentation']
        index_in_file = index%self.data_tensors['min_size_segmentation']
        if (self.data_cache is None) or self.data_cache[0] != index_file :
            file = self.file_list[index_file]
            tensors = torch.load(file, map_location=self.device)
            self.data_cache = (index_file, tensors)
        tensor = self.data_cache[1][index_in_file]
        label = self.labels[index]
        return tensor, label


In [4]:
# Définition des constantes et des hyper-paramètres

outputs_folder = "../data/saved_outputs/train"
common_subfolders_name = "weight_decay_"
models_folder = "../data/saved_models/various_weight_decay_models"
if not os.path.exists(models_folder):
    os.makedirs(models_folder)
    
input_size = 2048
num_classes = 1000

batch_size = 256
num_epochs = 90

num_weight_decay = 9
weight_decay_parameters = torch.logspace(-6,-2, num_weight_decay)
learning_rate = 0.1
momentum = 0.9
lr_decay_step = 30
decay_rate = 0.1


In [5]:
# Instanciation du dataset et du dataloader
dataset = PenultimateOutputsDataset(outputs_folder)
train_loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True)
len(dataset)

1281167

In [6]:
torch.manual_seed(0) # Reproductibilité de l'expérience

# Définition du modèle de régression logistique avec torch 
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
  
    def forward(self, x): 
        out = self.linear(x) 
        return out 

In [7]:
# Entraînement des modèles
for training_index, weight_decay_parameter in enumerate(weight_decay_parameters) :

    print(f"Processing step {training_index+1}/{len(weight_decay_parameters)}, weight decay = {weight_decay_parameter:.3e}.")
    
    # On définit le modèle
    model = LogisticRegression(input_size=input_size, num_classes=num_classes) 
    model = model.to(DEVICE)

    # Définition de la fonction de loss et de l'optimiseur
    loss_function = nn.CrossEntropyLoss() 
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay_parameter)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=lr_decay_step, gamma=decay_rate)

    # Définition du répertoire associé
    folder_path = os.path.join(models_folder, f"weight_decay_{weight_decay_parameter:.3e}")
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Initialisation des différentes variables pour stocker les éléments intéressants
    accuracy_history = []
    loss_history = torch.tensor([]).to(DEVICE) 
    complete_optimized_function_value_history = torch.tensor([]).to(DEVICE) 
    mean_loss_class = torch.tensor([]).to(DEVICE) 
    weights_epoch = torch.tensor([]).to(DEVICE) 

    # Obtention du nombre d'images par classe
    nb_image_class = torch.bincount(dataset.labels).to(DEVICE)
    
    for epoch in range(num_epochs): 

        # Définition des répertoires associés
        models_path = os.path.join(folder_path, "models")
        data_path = os.path.join(folder_path, "data")
        if not os.path.exists(models_path):
            os.makedirs(models_path)
        if not os.path.exists(data_path):
            os.makedirs(data_path)
            
        # Initialisation du nombre de prédictions correctes
        correct_predictions = 0
    
        # Initialisation du vecteur de la somme des erreurs pour chaque classe
        class_loss = torch.zeros(1000, device=DEVICE)
        
        for inputs, labels in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]"):
            
            # Chargement dans le bon appareil
            inputs = inputs.to(DEVICE) 
            labels = labels.to(DEVICE)
            
            # Forward pass 
            outputs = model(inputs) 
            loss = loss_function(outputs, labels)
            complete_optimized_function_value = loss + weight_decay_parameter * sum(t.pow(2).sum() for t in model.state_dict().values()) / 2
    
            # Calcul des erreurs pour chaque image et ajout dans class_loss
            input_loss = nn.functional.cross_entropy(outputs,labels,reduction='none') + weight_decay_parameter * sum(t.pow(2).sum() for t in model.state_dict().values()) / 2
            class_loss.index_add_(0,labels,input_loss)
            
            # Ajout des prédictions correctes au total
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
    
            # Backward and optimize 
            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step()
        
        # Modification du lerning rate
        scheduler.step()
    
        # Sauvegarde du modèle intermédiaire
        torch.save(model.state_dict(), f'{models_path}/epoch-{epoch}.pt')
    
        # Calcul de la précision
        accuracy = 100 * correct_predictions / len(dataset)
    
        # Sauvegarde des différentes mesures
        accuracy_history.append(accuracy)
        loss_history = torch.cat((loss_history, loss.unsqueeze(0)), dim=0)
        complete_optimized_function_value_history = torch.cat((complete_optimized_function_value_history, complete_optimized_function_value.unsqueeze(0)), dim=0)
        mean_loss_class = torch.cat((mean_loss_class, (class_loss/nb_image_class).detach().unsqueeze(0)), dim=0)
        weights_epoch = torch.cat((weights_epoch, sum(t.abs().sum() for t in model.state_dict().values()).unsqueeze(0)), dim=0)
        
        # Affichage
        print(f'>> Accuracy: {accuracy} %\n-- Loss: {loss.item()}\n-- Complete optimized function value: {complete_optimized_function_value.item()}')
            
    # Sauvegarde de toutes les données importantes liées à l'entraînement du modèle
    torch.save(accuracy_history, f'{data_path}/accuracy.pt')
    torch.save(loss_history, f'{data_path}/loss.pt')
    torch.save(complete_optimized_function_value, f'{data_path}/complete_optimized_function_values.pt')
    torch.save(mean_loss_class, f'{data_path}/mean_loss_class.pt')
    torch.save(weights_epoch, f'{data_path}/weights.pt')

    print(f"Step {training_index+1}/{len(weight_decay_parameters)} completed.")

Processing step 1/9, weight decay = 1.000e-06.


Epoch [1/90]: 100%|███████████████████████████████████████████████████████████████| 5005/5005 [00:36<00:00, 136.21it/s]


>> Accuracy: 8.454791607963676 %
-- Loss: 39.4916877746582
-- Complete optimized function value: 39.74224853515625


Epoch [2/90]: 100%|███████████████████████████████████████████████████████████████| 5005/5005 [00:36<00:00, 137.13it/s]


>> Accuracy: 61.90114169347166 %
-- Loss: 21.195581436157227
-- Complete optimized function value: 21.489025115966797


...

>> Accuracy: 56.191269366132595 %
-- Loss: 4.710214614868164
-- Complete optimized function value: 6.288237571716309


Epoch [89/90]: 100%|██████████████████████████████████████████████████████████████| 5005/5005 [00:41<00:00, 119.92it/s]


>> Accuracy: 56.191269366132595 %
-- Loss: 4.710214614868164
-- Complete optimized function value: 6.288237571716309


Epoch [90/90]: 100%|██████████████████████████████████████████████████████████████| 5005/5005 [00:39<00:00, 127.08it/s]

>> Accuracy: 56.191269366132595 %
-- Loss: 4.710214614868164
-- Complete optimized function value: 6.288237571716309
Step 9/9 completed.
